# Adding vertices to Entities in Document JSON

* Author: docai-incubator@google.com

## Disclaimer

This tool is not supported by the Google engineering team or product team. It is provided and supported on a best-effort basis by the **DocAI Incubator Team**. No guarantees of performance are implied.


## Objective

This document guides how to add vertices to an entity using its normalized vertices. This approach considers entity nesting as well.

## Prerequisites
* Vertex AI Notebook Or Colab (If using Colab, use authentication) 
* Permission for Vertex AI Notebook.


## Step by Step procedure 

### 1.Importing Required Modules

In [ ]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/document-ai-samples/main/incubator-tools/best-practices/utilities/utilities.py

In [ ]:
import json
from typing import List, Dict, Tuple, Optional
from google.cloud import storage
from google.cloud import documentai_v1beta3 as documentai

from utilities import blob_downloader, store_document_as_json, file_names

### 2.Setup the inputs

* `file_path` : Location of the Json File in the Google Cloud Storage(GCS) Bucket
* `output_path` : The Google Cloud Storage(GCS) path for updated labeled jsons to store

In [ ]:
file_path = "gs://bucket_name/input_sub_folder_path/"
output_path = "gs://bucket_name/output_sub_folder_path/"

### 3.Run the required functions

In [ ]:
def normalized_to_original(
    normalized_vertices: List[Dict[str, float]], page_width: int, page_height: int
) -> List[Dict[str, int]]:
    """
    Convert normalized vertices (with values between 0 and 1) to original pixel-based vertices using the page's width and height.

    Args:
        normalized_vertices (List[Dict[str, float]]): A list of dictionaries representing the normalized vertices where
                                                      'x' and 'y' values are between 0 and 1.
        page_width (int): The width of the page in pixels.
        page_height (int): The height of the page in pixels.

    Returns:
        List[Dict[str, int]]: A list of dictionaries representing the original vertices with 'x' and 'y' values converted
                              to pixel coordinates based on the page dimensions.
    """
    original_vertices = []
    for vertex in normalized_vertices:
        original_vertex = {
            "x": round(vertex["x"] * page_width),
            "y": round(vertex["y"] * page_height),
        }
        original_vertices.append(original_vertex)

    return original_vertices


def extract_page_dimensions(document: dict, page_index: int) -> Tuple[float, float]:
    """
    Extract the width and height of a page at the given index from the document.

    Args:
        document (dict): A dictionary representing the document, which includes a list of pages.
        page_index (int): The index of the page from which to extract the dimensions.

    Returns:
        Tuple[float, float]: The width and height of the page as a tuple of float values.

    Raises:
        ValueError: If the page does not have width and height dimensions.
    """
    page = document.get("pages", [])[page_index]
    dimension = page.get("dimension", {})
    width = dimension.get("width")
    height = dimension.get("height")

    if width is not None and height is not None:
        return width, height
    else:
        raise ValueError(f"Page {page_index} dimensions not found.")


def process_entity(entity: dict, document: dict, page_index: int) -> Optional[None]:
    """
    Process a single entity by converting its normalized vertices to original pixel-based vertices.

    Args:
        entity (dict): A dictionary representing the entity, which includes page references and bounding polygon details.
        document (dict): The entire document containing pages, dimensions, and other metadata.
        page_index (int): The index of the page from which the entity is extracted.
                          Used as a default if no page information is provided in the entity.

    Returns:
        None: The function modifies the entity in place by adding original vertices based on page dimensions.
              If the entity lacks normalized vertices or page information, it returns None to skip further processing.
    """
    entity_page_refs = entity.get("pageAnchor", {}).get("pageRefs", [])

    # Use page 0 as default if no page information is present and assume single page document
    if not entity_page_refs:
        entity_page_ref = {"page": page_index}
        entity_page_refs = [entity_page_ref]

    # Use the first page reference (assuming single-page entities)
    entity_page_ref = entity_page_refs[0]
    page_index = int(
        entity_page_ref.get("page", page_index)
    )  # Default to page_index if page info is missing
    entity_bounding_poly = entity_page_ref.get("boundingPoly", {})
    entity_normalized_vertices = entity_bounding_poly.get("normalizedVertices", [])

    if not entity_normalized_vertices:
        return  # Skip if there are no normalized vertices for the entity

    # Get the dimensions of the page where this entity is located
    try:
        page_width, page_height = extract_page_dimensions(document, page_index)
    except ValueError as e:
        print(e)
        return

    # Convert normalized vertices to original vertices (in pixel coordinates)
    entity_bounding_poly["vertices"] = normalized_to_original(
        entity_normalized_vertices, page_width, page_height
    )


def handle_nested_entities(
    entities: List[Dict], document: Dict, page_index: int = 0
) -> None:
    """
    Recursively process a list of entities and their nested entities by converting
    normalized vertices to original pixel-based vertices.

    Args:
        entities (List[Dict]): A list of dictionaries representing entities, each potentially containing
                               properties (nested entities) and bounding polygon details.
        document (Dict): The document structure that includes page dimensions and other metadata.
        page_index (int, optional): The index of the page to be used as a fallback if no page information
                                    is provided for an entity. Defaults to 0.

    Returns:
        None: The function processes each entity in place, handling both parent and nested entities.
    """
    for entity in entities:
        # Process the current entity
        process_entity(entity, document, page_index)

        # Check for nested entities and process them recursively
        nested_entities = entity.get("properties", [])
        if nested_entities:
            handle_nested_entities(nested_entities, document, page_index)


def add_vertices_to_entities(document: Dict) -> Dict:
    """
    Adds original pixel-based vertices to all entities in the provided document by converting
    their normalized vertices using the document's page dimensions.

    Args:
        document (Dict): A dictionary representing the document, which contains entity data,
                         page information, and normalized vertices.

    Returns:
        Dict: The modified document where all entities (including nested ones) are updated
              with their original pixel-based vertices.
    """
    page_index = 0  # Default to page 0 for single-page documents

    # Retrieve all entities from the document
    entities = document.get("entities", [])

    # Process the entities if any are present
    if entities:
        handle_nested_entities(entities, document, page_index)

    return document

### 4.Run the code

In [ ]:
def main():
    file_name_list, file_path_dict = file_names(file_path)
    print(file_path)
    for i in range(len(file_name_list)):
        json_file_path = (
            "gs://" + file_path.split("/")[2] + "/" + file_path_dict[file_name_list[i]]
        )
        print(json_file_path)
        document_data = blob_downloader(
            json_file_path.split("/")[2], "/".join(json_file_path.split("/")[3:])
        )
        # print(document_data)

        updated_document = add_vertices_to_entities(document_data)

        # Save the updated JSON
        print(output_path)
        store_document_as_json(
            json.dumps(updated_document),
            output_path.split("/")[2],
            ("/").join(output_path.split("/")[3:]) + "/" + file_name_list[i],
        )

    print(f"Vertices added to entities. Output file saved to {output_path}")


main()

### 5.Output

Compare the difference between before and after updated json file

#### Before Updating the Json File
<img src="./Images/before_updation.png" width=800 height=400 ></img>
#### After Updating the Json File
<img src="./Images/after_updation.png" width=800 height=400 ></img>